#Прохождение аутентификации ВК

In [ ]:
get_ipython().system('pip install vk_api')

In [ ]:
# Для прохождения 2х факторной аутентификации
def auth_handler():
    key = input("Enter authentication code: ")
    remember_device = False

    return key, remember_device


In [ ]:
import vk_api
login = '' #Моя почта
password = '' #Мой пароль от ВК

vk_session = vk_api.VkApi(login, password, auth_handler=auth_handler) # auth_handler - Нужно ввести свой код из вк.
vk = vk_session.get_api()
vk_session.auth()

# Парсинг постов

In [ ]:
posts_list = []
posts = None

# Максимальное кол-во постов, которое можно загрузить за раз = 100. Особенность API ВК
# Для преодоления ограниченя используется цикл
for i in range(50):
  local_posts = vk.wall.get(domain='trashbox_ru', count=100, offset=100*i) # domain='' - источник новостей. offset - кол. постов для парсинга.
  if posts is None:
    posts = local_posts
  else:
    posts['items'].extend(local_posts['items'])
  posts_list.extend(local_posts['items'])

print("Прочитано: {} постов".format(posts["count"]))
print(posts_list[0]) # Вывод инфы 1 поста, что был скопирован

# Группы, что я парсил:
# https://vk.com/trashbox_ru - Высокие технологии
# https://vk.com/rsportru - Спорт
# https://vk.com/news.politics - Политика

# Трансформация постов


In [ ]:
from datetime import datetime
from typing import Dict, Any
import re # Для регулярных выражений

def transform(post: Dict[str, Any]) -> Dict[str, Any]:
  result = {}

  # Получение текста поста
  post_text = post.get('text')

  # Удаление ненужных нейросети данных с помощью регулярных выражений

  # Удаление ссылок
  post_text = re.sub(r'(https?://[^\"\s>]+)','',post_text)
  
  # Удаление Символов Хештег
  post_text = re.sub(r'#','',post_text)

  # Удаление смайликов # Не ловит все смайлики!
  post_text = re.sub(r'(\u00a9|\u00ae|[\u2000-\u3300]|\ud83c[\ud000-\udfff]|\ud83d[\ud000-\udfff]|\ud83e[\ud000-\udfff])','',post_text)

  # Сохранение обработанного текста поста
  result['text'] = post_text

  return result

In [ ]:
# Тест работы рег. выражений исп. в функции

text = 'Обычный текст Ссылка https://yandex.ru/ Хештег #Учись ☺'

# Удаление ссылок
text = re.sub(r'(https?://[^\"\s>]+)','',text)
  
# Удаление Символов Хештег
text = re.sub(r'#','',text)

# Удаление смайликов
text = re.sub(r'(\u00a9|\u00ae|[\u2000-\u3300]|\ud83c[\ud000-\udfff]|\ud83d[\ud000-\udfff]|\ud83e[\ud000-\udfff])','',text)

print(text)

Обычный текст Ссылка  Хештег Учись 


In [ ]:
# Тест на обработку поста

# 1 пост до трансформации
print(posts_list[0])

print("\n")

# 1 пост после трансформации
print(transform(posts_list[0]))

{'id': 121382, 'from_id': -5264590, 'owner_id': -5264590, 'date': 1637247516, 'marked_as_ads': 0, 'post_type': 'post', 'text': '🤔 И снова рубрика «Apple такого никогда не сделает» — прозрачный кейс для AirPods Pro\n\nПочему бы и нет?', 'attachments': [{'type': 'photo', 'photo': {'album_id': -7, 'date': 1637247516, 'id': 457254845, 'owner_id': -5264590, 'has_tags': False, 'access_key': '1717997cca5d491dc5', 'post_id': 121382, 'sizes': [{'height': 130, 'url': 'https://sun9-8.userapi.com/impg/S1mEwyzLfEHTfEE7swK38_F2nDZfqD3TLUI9DQ/96lLwEigJS8.jpg?size=112x130&quality=96&sign=ded43c52db03609019f50a387a5dc1f0&c_uniq_tag=2ijbeQrmTIustoMBk7YaFWbisNP9yfTLF_UdZBmIhSc&type=album', 'type': 'm', 'width': 112}, {'height': 150, 'url': 'https://sun9-8.userapi.com/impg/S1mEwyzLfEHTfEE7swK38_F2nDZfqD3TLUI9DQ/96lLwEigJS8.jpg?size=130x150&quality=96&sign=fef3b10760bd2d143850898730649236&c_uniq_tag=vFSqWZmYyzNWK1cQB8l8JLJvRr1Dyxvcfbv-2o-OSOU&type=album', 'type': 'o', 'width': 130}, {'height': 231, 'url': 

In [ ]:
# Трансформация всех постов
preprocessed_posts = list(map(lambda x: transform(x), posts_list))
preprocessed_posts

[{'text': '🤔 И снова рубрика «Apple такого никогда не сделает»  прозрачный кейс для AirPods Pro\n\nПочему бы и нет?'},
 {'text': '📊 TikTok больше не рекордсмен по скачиваниям, а Tinder прибыльнее YouTube\n\nВ октябре 2021 года глобальные загрузки Instagram увеличились до 53 млн  соцсеть вышла на 1 место в рейтинге. TikTok теперь на 2 месте с 44 млн установок. Между тем, в США Tinder приносит 85 млн долларов, что больше, чем у крупнейших стриминговых сервисов и даже видеохостинга Google.'},
 {'text': ''},
 {'text': '️ Больше Material You: в Gmail на Android 12 появились новые удобные виджеты\n\nТри размера, округлые формы и много кнопок  теперь они такие. Можно быстро создать письмо, отправить входящие в архив, а в большом варианте даже открыть чаты, групповые переписки и встречи. Всё в одно нажатие.'},
 {'text': '🤖 6 пицц и 4 литра газировки: новые вместительные роверы-курьеры Яндекса уже на дорогах\n\nТретье поколение роботов может перевозить до 20 кг груза  теперь в них помещаются и 

# Сохранение трансформированных данных в файл

In [ ]:
import pandas as pd

# Сохранение результатов парсинга в csv файл
df = pd.DataFrame.from_dict(preprocessed_posts)
df.to_csv('groups.csv')

# Сохранение из csv в txt
with open('out.txt', 'a') as file:
  file.write('\n'.join(df['text'].values))

# Задание 7

In [ ]:
# import string
# from typing import List


# punctuation_tokens = string.punctuation  # получаем все знаки пунктуации

# # Добавляем в знаки пунктуации знак переноса строки
# # Это можно удалить, для этого нужно удалить две нижние строки
# punctuation_tokens = list(punctuation_tokens) # это удалить
# punctuation_tokens.append('\n') # это удалить
# def get_wordlist(texts: List[str]) -> List[str]:
#     all_words = []
#     # Перебираем тексты всех постов
#     for t in texts:
#       text = t
#       # Перебираем все знаки пунктуации
#       for p in punctuation_tokens:
#         # Удаляем из текста знак пунктуации
#         text = text.replace(p, '')
#       no_digit = []
#       # Перебираем каждое слово текста
#       for d in text.split(' '):
#         # Проверяем является ли оно числом, если да, то не записываем его
#         if not d.isdigit():
#           no_digit.append(d)
#       # В список со всеми словами, записываем слова без знаков пунктуации и те, что не являются числом
#       all_words.extend(no_digit)
#     return all_words
# # df['text'].values все значения колонки text списком
# all_words = get_wordlist(df['text'].values)
# print(all_words)

# Задание 8

In [ ]:
# from collections import Counter
# # Из документации 
# Counter(all_words).most_common(20)

# Задание 9

In [ ]:
# counter = 0
# # Начинаем с первого поста
# # Переменная posts_list - была заполнена в первом задании

# posts_counter = 0
# all_comments = []
# # Перебираем все посты, пока не соберём 100 комментариев
# while len(all_comments) < 100:
#   # Получаем пост по его id, далее получаем все комментарии поста
#   comments = vk.wall.getComments(count=100, owner_id=posts_list[posts_counter].get('owner_id'), post_id=posts_list[posts_counter].get('id'))
#   # Достаём все комментарии из поста
#   # Максимальное кол-во комментарие = 100
#   comments = comments.get('items')
#   # Нажняя запись аналогична:
#   # def func(x):
#     # return x['text']
#   # all_comments = []
#   # for p in comments:
#     # all_comments.append(func(x))
#   all_comments.extend(list(map(lambda x: x['text'], comments)))
#   # Увеличиваем счётчик постов на один, чтобы забрать комментарии со следующего поста
#   posts_counter += 1

# # Тут всё как в задание 7 и 8
# all_comments_word_list = get_wordlist(all_comments)
# Counter(all_comments_word_list).most_common(20)